In [1]:
# Импорт библиотек
import os
import time
import cotel
import Turboustanovka
import GTU
import mat_properties as prop
import numpy as n
import pandas as pd
from scipy.optimize import root


# таблица номинального режима
gas_streams0 = pd.read_excel("streams0.xlsx", sheet_name="gas", index_col=0)
water_streams0 = pd.read_excel(
    "streams0.xlsx", sheet_name="water", index_col=0)
GTU_ISO = pd.read_excel("input.xlsx", sheet_name="ISO", index_col=0)
GTU_input = pd.read_excel("input.xlsx", sheet_name="GTU_input", index_col=0)
# рабочая таблица (=номинал в 1 итерации)
gas_streams = pd.read_excel("streams.xlsx", sheet_name="gas", index_col=0)
water_streams = pd.read_excel("streams.xlsx", sheet_name="water", index_col=0)
# рабочая таблица показателей блоков
heaters = pd.read_excel("blocks.xlsx", sheet_name="heaters", index_col=0)
electric = pd.read_excel("blocks.xlsx", sheet_name="electric", index_col=0)

# Состав газов в номинале
gasmix = "Nitrogen*Oxygen*CO2*Water*Argon"
# Считывание рефпропа
RP = prop.init_REFPROP(r"C:\Program Files (x86)\REFPROP")
fractiongas0 = (
    gas_streams0.at["GTU-PEVD", "N2"],
    gas_streams0.at["GTU-PEVD", "O2"],
    gas_streams0.at["GTU-PEVD", "CO2"],
    gas_streams0.at["GTU-PEVD", "H2O"],
    gas_streams0.at["GTU-PEVD", "Ar"],
)

gas0 = prop.Materials_prop(
    gasmix,
    fractiongas0,
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    RP=RP,
)
water = prop.Materials_prop(
    "water",
    [1.0, 0, 0, 0, 0],
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    RP=RP,
)


# Задание энтальпий газа в номинальном режиме
Temperatures = gas_streams0.loc["GTU-KU":"GPK-out", "T"]
Pressure = gas_streams0.loc["GTU-KU", "P"]
Enthalpies = list(map(lambda x: gas0.p_t(Pressure, x)["h"], Temperatures))
gas_streams0.loc["GTU-KU":"GPK-out", "H"] = Enthalpies


####################################################################################################
# Расчет ГТУ
Gas_turbine = GTU.gtu(GTU_ISO, GTU_input, "GTU-KU")
Gas_turbine_res = Gas_turbine.calc()
electric.at["GTU", "N"] = Gas_turbine_res["N"]
electric.at["GTU", "KPD"] = Gas_turbine_res["eff"]
electric.at["DK", "N"] = Gas_turbine_res["Ndk"]
gas_streams = pd.read_excel("streams.xlsx", sheet_name="gas", index_col=0)
gas_streams.at["GTU-KU", "T"] = Gas_turbine_res["T"]
gas_streams.at["GTU-KU", "G"] = Gas_turbine_res["G"]
gas_streams.at["GTU-KU", "P"] = 0.1
gas_streams.at['GTU-KU', 'H'] = gas0.p_t(
    gas_streams.at['GTU-KU', 'P'], gas_streams.at['GTU-KU', 'T'])['h']
Gas_turbine_composition = pd.read_excel(
    "input.xlsx", sheet_name="Gas_composition0", index_col=0)

# Параметры газа на выходе в КУ
gas_streams.loc["GTU-PEVD", "T":"P"] = gas_streams.loc["GTU-KU", "T":"P"]
gas_streams.at["GTU-PEVD", "G"] = gas_streams.loc["GTU-KU", "G"]
gas_streams.at["GTU-PEVD",
               "N2":"Ar"] = Gas_turbine_composition.loc["Fraction", "N2":"Ar"]

# Состав газов при частичной нагрузке
fractiongas = (
    gas_streams.at["GTU-PEVD", "N2"],
    gas_streams.at["GTU-PEVD", "O2"],
    gas_streams.at["GTU-PEVD", "CO2"],
    gas_streams.at["GTU-PEVD", "H2O"],
    gas_streams.at["GTU-PEVD", "Ar"],
)
gas1 = prop.Materials_prop(
    gasmix,
    fractiongas,
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    RP=RP,
)

# Основные константы
Calctolerance = 10**-3
Calcmethod = "hybr"
KPDnasos = 0.8074
KPD = 0.99
Maxiterations = 10

####################################################################################################
# Инициализируем все
TU = Turboustanovka.turboustanovka(
    water, water_streams0, water_streams, heaters, electric, KPD_SP=0.99, KPDnasos=0.8)
Whole_cotel = cotel.cotel_all(KPD, KPDnasos,  gas0, gas1, water, Calcmethod,
                              gas_streams0, water_streams0, gas_streams, water_streams, heaters, electric)


Vvd0 = 1/water.p_h(water_streams0.at["PEVD-DROSVD", 'P'],
                   water_streams0.at["PEVD-DROSVD", 'H'])['rho']
Vnd0 = 1/water.p_h(water_streams0.at["PPND-DROSND", 'P'],
                   water_streams0.at["PPND-DROSND", 'H'])['rho']
Gvd0 = water_streams0.at["PEVD-DROSVD", 'G']
Gnd0 = water_streams0.at["PPND-DROSND", 'G']
dPvd0 = water_streams0.at["PEVD-DROSVD", 'P'] - \
    water_streams0.at["DROSVD-TURBVD", 'P']
dPnd0 = water_streams0.at["PPND-DROSND", 'P'] - \
    water_streams0.at["DROSND-TURBND", 'P']

####################################################################################################
# Расчет всего
Pnd_it = []
Pvd_it = []

start_time = time.time()
Max_error_P = 2
Max_error_G = 2
Max_error = 2
Error_nd_P = 2
Error_vd_P = 2

for i in range(Maxiterations):

    for j in range(Maxiterations):
        if Max_error > 0.01:
            Calctolerance_new = 10**-2
        if Max_error > 0.1:
            Calctolerance_new = 10**-1
        if Max_error > 1:
            Calctolerance_new = 10**-1
        else:
            Calctolerance_new = Calctolerance

        Whole_cotel.calc(Calctolerance_new, Maxiterations)
        print('КУ расчитан')

        Gnd1 = water_streams.at["PPND-DROSND", "G"]
        Gnd2 = water_streams.at["DROSND-TURBND", "G"]
        Gvd1 = water_streams.at["PEVD-DROSVD", "G"]
        Gvd2 = water_streams.at["DROSVD-TURBVD", "G"]

        # Перекидываем расходы
        water_streams.at["DROSVD-TURBVD", 'G'] = Gvd1
        water_streams.at["DROSND-TURBND", 'G'] = Gnd1

        # Перекидываем энтальпии
        water_streams.at["DROSVD-TURBVD",
                         'H'] = water_streams.at["PEVD-DROSVD", 'H']
        water_streams.at["DROSND-TURBND",
                         'H'] = water_streams.at["PPND-DROSND", 'H']

        TU_res = TU.calculate(
            teplofikacia=0, calcmethod="hybr", calctolerance=Calctolerance, maxiterations=Maxiterations
        )
        print('ТУ расчитана')

#         if i%3==0:
#             print(water_streams.loc['PEVD-DROSVD','T':"G"]+water_streams.loc['PPND-DROSND','T':"G"])

        # основные потоки, которые надо сводить по расходу
        Error_water_G = abs((water_streams.at["SMESHOD-REC", "G"] -
                             water_streams.at["GPK-IND", "G"])/water_streams.at["GPK-IND", "G"]*100)
        Error_nd_G = abs((Gnd1 - Gnd2)/Gnd1*100)
        Error_vd_G = abs((Gvd1 - Gvd2)/Gvd1*100)
        Max_error_G = max(Error_water_G, Error_nd_G, Error_vd_G)
        print('Error_water_G', Error_water_G)
        print('Error_nd_G', Error_nd_G)
        print('Error_vd_G', Error_vd_G)
        print('Gnd1', Gnd1, 'Gnd2', Gnd2)
        print('Gvd1', Gvd1, 'Gvd2', Gvd2)
        Max_error = max(Error_water_G, Error_nd_G,
                        Error_vd_G, Error_nd_P, Error_vd_P)
        if abs(Max_error_G) < Calctolerance_new:
            print(
                "Максимальная погрешность определения расхода в КУ+ПТУ", Max_error_G)
            break
        if i < 2:
            print("Первые сведения расходов")
        if j == Maxiterations - 1:
            print("Достигнуто максимальное количество итераций расхода КУ+ПТУ")

    Pnd_it.append(round(water_streams.loc['PPND-DROSND', 'P'], 5))
    Pvd_it.append(round(water_streams.loc['PEVD-DROSVD', 'P'], 5))
    # Переписываю давления
    P_turb_vd = water_streams.at["DROSVD-TURBVD", 'P']
    P_turb_nd = water_streams.at["DROSND-TURBND", 'P']
    Vvd1 = 1/water.p_h(water_streams0.at["PEVD-DROSVD", 'P'],
                       water_streams0.at["PEVD-DROSVD", 'H'])['rho']
    Vnd1 = 1/water.p_h(water_streams0.at["PPND-DROSND", 'P'],
                       water_streams0.at["PPND-DROSND", 'H'])['rho']
    dPvd = dPvd0 * (Gvd1*Vvd1/Gvd0/Vvd0)**2*Vvd0/Vvd1
    dPnd = dPnd0*(Gnd1*Vnd1/Gnd0/Vnd0)**2*Vnd0/Vnd1
    P_kotel_vd = water_streams.at["PEVD-DROSVD", 'P']
    P_kotel_nd = water_streams.at["PPND-DROSND", 'P']
    P_kotel_nd_new = P_turb_nd+dPnd
    P_kotel_vd_new = P_turb_vd+dPvd
    water_streams.at["PPND-DROSND", 'P'] = (P_kotel_nd_new+P_kotel_nd)/2
    water_streams.at["PEVD-DROSVD", 'P'] = (P_kotel_vd_new+P_kotel_vd)/2

    print('Pnd_it', Pnd_it)
    print('Pvd_it', Pvd_it)

    Error_nd_P = abs((P_kotel_nd - P_kotel_nd_new)/P_kotel_nd*100)
    Error_vd_P = abs((P_kotel_vd - P_kotel_vd_new)/P_kotel_vd*100)
    Max_error = max(Error_water_G, Error_nd_G,
                    Error_vd_G, Error_nd_P, Error_vd_P)

    print('Error_nd_P', Error_nd_P)
    print('Error_vd_P', Error_vd_P)
    Max_error_P = max(Error_nd_P, Error_vd_P)
    if abs(Max_error) < Calctolerance and Calctolerance_new == Calctolerance:
        print(
            "Максимальная погрешность определения расходов", Error_water_G)
        break
    if i == Maxiterations - 1:
        print("Достигнуто максимальное количество итераций давления КУ+ПТУ", i+1)

print(
    "Степень сухости пара в ЭВД: ",
    water.p_h(water_streams.at["EVD-IVD", "P"],
              water_streams.at["EVD-IVD", "H"])["Q"],
)
print(
    "Степень сухости пара в ГПК: ",
    water.p_h(water_streams.at["GPK-IND", "P"],
              water_streams.at["GPK-IND", "H"])["Q"],
)

print("fin ALL:--- %s сек. ---" % round((time.time() - start_time), 1))
print(Error_water_G)

Достигнуто максимальное количество итераций контура ГПК
dQ/Qsumm 2.361512287426756
dQ/Qsumm -0.019570194545250084
Fin:--- 23.32 сек. ---
dQ/Qsumm -0.019570194545250084
dQ/Qvd -0.05137386771956573
dQ/Qnd 0.005629819292010887
КУ расчитан
ТУ расчитана
Error_water_G 0.0
Error_nd_G 15.64064475383273
Error_vd_G 5.724386668791735
Gnd1 12.529730661354517 Gnd2 10.57
Gvd1 39.697558266742746 Gvd2 41.97
Первые сведения расходов
dQ/Qsumm 0.030692162166309177
Fin:--- 3.34 сек. ---
dQ/Qsumm 0.030692162166309177
dQ/Qvd 0.014550732989049731
dQ/Qnd 0.06667696854479546
КУ расчитан
ТУ расчитана
Error_water_G 0.0
Error_nd_G 0.06336426671032507
Error_vd_G 0.04688573105645483
Gnd1 12.537675067223386 Gnd2 12.529730661354517
Gvd1 39.6789544988504 Gvd2 39.697558266742746
Максимальная погрешность определения расхода в КУ+ПТУ 0.06336426671032507
Pnd_it [0.691]
Pvd_it [8.407]
Error_nd_P 32.632193077308344
Error_vd_P 39.96017859674229
dQ/Qsumm -1.4020049861766213
Достигнуто максимальное количество итераций контура 

In [2]:
# Вывод газовых потоков
gas_streams

,T,P,H,G,N2,O2,CO2,H2O,Ar
AIR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GTU-KU,509.892416,0.1,921.662785,354.05446,0.7803,0.1237,0.0301,0.0594,0.0065
GTU-PEVD,509.892416,0.1,921.662785,354.05446,0.7803,0.1237,0.0301,0.0594,0.0065
PEVD-IVD,446.093303,0.1,849.222533,354.05446,0.7803,0.1237,0.0301,0.0594,0.0065
IVD-EVD,275.623088,0.1,660.916400,354.05446,0.7803,0.1237,0.0301,0.0594,0.0065
EVD-PPND,221.255101,0.1,602.374752,354.05446,0.7803,0.1237,0.0301,0.0594,0.0065
PPND-IND,218.334077,0.1,599.247959,354.05446,0.7803,0.1237,0.0301,0.0594,0.0065
IND-GPK,157.085230,0.1,534.074043,354.05446,0.7803,0.1237,0.0301,0.0594,0.0065
GPK-out,96.451780,0.1,470.183478,354.05446,0.7803,0.1237,0.0301,0.0594,0.0065


In [3]:
# Вывод водяных потоков
water_streams

,T,P,H,G,S,X
AIR,NaN,NaN,NaN,NaN,NaN,NaN
PEVD-DROSVD,489.309764,5.310374,3405.952238,41.156222,NaN,NaN
IVD-PEVD,270.547436,5.550262,2789.234159,41.156222,NaN,NaN
EVD-IVD,270.058031,5.550262,1185.550780,41.156222,NaN,NaN
PEN-EVD,161.986107,5.550262,686.948961,41.156222,NaN,NaN
BND-PEN,147.059448,0.439799,619.502872,41.156222,NaN,NaN
PPND-DROSND,193.613530,0.435106,2845.602603,10.614802,NaN,NaN
IND-PPND,147.059448,0.439799,2742.351309,10.614802,NaN,NaN
GPK-IND,145.728773,0.439799,613.783886,51.771024,NaN,NaN
GPK-REC,145.728773,0.439799,613.783886,61.820498,NaN,NaN


In [4]:
Errors_water = abs(water_streams - water_streams0)/water_streams0*100
Errors_gas = abs(gas_streams-gas_streams0)/gas_streams0*100
Errors_gas
Errors_water

,G,H,P,S,T,Unnamed: 6,Unnamed: 7,X
AIR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BND-PEN,35.084824,11.003187,36.846743,NaN,10.731510,NaN,NaN,NaN
DOOTB1,33.762764,0.313203,34.374335,2.330133,NaN,NaN,NaN,NaN
DOOTB2,33.762764,0.351155,34.269852,2.339050,6.582540,NaN,NaN,NaN
DROSND-TURBND,28.083995,0.899356,35.622385,2.048721,8.225236,NaN,NaN,NaN
DROSVD-TURBVD,35.084824,0.484240,35.351368,NaN,4.034933,NaN,NaN,NaN
ENDOFVD,35.084824,0.277072,34.192829,NaN,4.348762,NaN,NaN,NaN
EVD-IVD,35.084824,11.390387,36.539427,NaN,9.619133,NaN,NaN,NaN
GPK-BYPASS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GPK-IND,33.762764,11.748462,36.846743,NaN,11.464901,NaN,NaN,NaN


In [5]:
# Невязка по всем параметрам
Qgas = (
    KPD
    * gas_streams.at["GTU-PEVD", "G"]
    * (gas_streams.at["GTU-PEVD", "H"] - gas_streams.at["PEVD-IVD", "H"])
)
Qwat = water_streams.at["PEVD-DROSVD", "G"] * (
    water_streams.at["PEVD-DROSVD", "H"] - water_streams.at["IVD-PEVD", "H"]
)
print("dQ/Q ПЕВД+ИВД", ((Qgas - Qwat) / Qgas * 100))

Qgas1 = (
    KPD
    * gas_streams.at["GTU-PEVD", "G"]
    * (gas_streams.at["GTU-PEVD", "H"] - gas_streams.at["IVD-EVD", "H"])
)
Qwat1 = water_streams.at["PEVD-DROSVD", "G"] * (
    water_streams.at["PEVD-DROSVD", "H"] - water_streams.at["EVD-IVD", "H"]
)
print("dQ/Q ПЕВД+ИВД+ЭВД", ((Qgas1 - Qwat1) / Qgas1 * 100))

Qgas = (
    KPD
    * gas_streams.at["EVD-PPND", "G"]
    * (gas_streams.at["EVD-PPND", "H"] - gas_streams.at["IND-GPK", "H"])
)
Qwat = (
    water_streams.at["IND-PPND", "G"]
    * (water_streams.at["IND-PPND", "H"] - water_streams.at["GPK-IND", "H"])
    + water_streams.at["PPND-DROSND", "G"]
    * (water_streams.at["PPND-DROSND", "H"] - water_streams.at["IND-PPND", "H"])
    + water_streams.at["BND-PEN", "G"]
    * (water_streams.at["BND-PEN", "H"] - water_streams.at["GPK-IND", "H"])
)
print("dQ/Q ППНД+ИНД", ((Qgas - Qwat) / Qgas * 100))

# Qgas1 = (
#     KPD
#     * gas_streams.at["EVD-PPND", "G"]
#     * (gas_streams.at["EVD-PPND", "H"] - gas_streams.at["GPK-out", "H"])
# )
# Qwat1 = (
#     water_streams.at["GPK-IND", "G"]
#     * (water_streams.at["GPK-IND", "H"] - water_streams.at["X-GPK", "H"])
#     + water_streams.at["IND-PPND", "G"]
#     * (water_streams.at["IND-PPND", "H"] - water_streams.at["GPK-IND", "H"])
#     + water_streams.at["PPND-DROSND", "G"]
#     * (water_streams.at["PPND-DROSND", "H"] - water_streams.at["IND-PPND", "H"])
#     + water_streams.at["BND-PEN", "G"]
#     * (water_streams.at["BND-PEN", "H"] - water_streams.at["GPK-IND", "H"])
# )
# print("dQ/Q ППНД+ИНД+ГПК", (Qgas1 - Qwat1) / Qgas1 * 100)

# Qgasall = (
#     KPD
#     * gas_streams.at["GTU-PEVD", "G"]
#     * (gas_streams.at["GTU-PEVD", "H"] - gas_streams.at["GPK-out", "H"])
# )
# Qwatall = (
#     water_streams.at["PPND-DROSND", "G"]
#     * (water_streams.at["PPND-DROSND", "H"] - water_streams.at["X-GPK", "H"])
#     + water_streams.at["PEVD-DROSVD", "G"]
#     * (water_streams.at["PEVD-DROSVD", "H"] - water_streams.at["X-GPK", "H"])
#     - water_streams.at["BND-PEN", "G"]
#     * (water_streams.at["PEN-EVD", "H"] - water_streams.at["BND-PEN", "H"])
# )
# print("dQ/Qsumm", (Qgasall - Qwatall) / Qgasall * 100)

dQ/Q ПЕВД+ИВД 0.037535158608208635
dQ/Q ПЕВД+ИВД+ЭВД 0.013022615945128331
dQ/Q ППНД+ИНД 0.06124556390158591


In [6]:
gas_streams0

,T,P,H,G,N2,O2,CO2,H2O,Ar
GTU-KU,542.10,0.1,958.637030,503.8,0.7803,0.1237,0.0301,0.0594,0.0065
GTU-PEVD,542.10,0.1,958.637030,503.8,0.7803,0.1237,0.0301,0.0594,0.0065
PEVD-IVD,468.05,0.1,874.031830,503.8,0.7803,0.1237,0.0301,0.0594,0.0065
IVD-EVD,309.20,0.1,697.417326,503.8,0.7803,0.1237,0.0301,0.0594,0.0065
EVD-PPND,235.90,0.1,618.078812,503.8,0.7803,0.1237,0.0301,0.0594,0.0065
PPND-IND,233.00,0.1,614.965387,503.8,0.7803,0.1237,0.0301,0.0594,0.0065
IND-GPK,175.50,0.1,553.594555,503.8,0.7803,0.1237,0.0301,0.0594,0.0065
GPK-out,106.90,0.1,481.155410,503.8,0.7803,0.1237,0.0301,0.0594,0.0065
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
water_streams0

,T,P,H,G,S,Unnamed: 6,Unnamed: 7
PEVD-DROSVD,511.500000,8.407000,3423.209978,63.400000,NaN,NaN,NaN
IVD-PEVD,301.297556,8.746000,2747.104023,63.400000,NaN,NaN,NaN
EVD-IVD,298.800000,8.746000,1337.948265,63.400000,NaN,NaN,NaN
PEN-EVD,166.500000,8.746000,708.319849,63.400000,NaN,NaN,NaN
BND-PEN,164.738361,0.696400,696.095567,63.400000,NaN,NaN,NaN
PPND-DROSND,211.600000,0.691000,2871.481837,14.760000,NaN,NaN,NaN
IND-PPND,164.738361,0.696400,2762.534980,14.760000,NaN,NaN,NaN
GPK-IND,164.600000,0.696400,695.493701,78.160000,NaN,NaN,NaN
GPK-REC,164.600000,0.696400,695.493701,82.910000,NaN,NaN,NaN
REC-GPK,60.000000,0.696400,251.748396,82.910000,NaN,NaN,NaN


In [8]:
electric

,Ni,N,KPDm,KPD
PEN,2.775826,3.434614,0.808192,0.082279
CVD,NaN,NaN,NaN,NaN
CND,NaN,NaN,NaN,NaN
GTU,NaN,82.830000,NaN,28.111134
DK,NaN,6.125634,NaN,NaN
Turbine,47.470326,NaN,NaN,NaN
Tots1,23.440437,NaN,NaN,NaN
Tots2,6.367696,NaN,NaN,NaN
Tots3,13.955429,NaN,NaN,NaN
Tots4,3.706764,NaN,NaN,NaN


In [9]:
heaters

,Qw,Qg,KPD
PEVD,25391.316563,25647.794508,0.99
IVD,66003.919754,66670.626014,0.99
EVD,20519.662475,20726.931793,0.99
PPND,1095.984280,1107.054828,0.99
IND,22844.364681,23075.115839,0.99
GPK,22394.531859,22620.739251,0.99
SP2,NaN,NaN,NaN
SP1,NaN,NaN,NaN
OD,NaN,NaN,NaN


In [10]:
GTU_input

,1,2,3,4,5,6
tair,-40.0,NaN,NaN,NaN,NaN,NaN
n,0.5,NaN,NaN,NaN,NaN,NaN
Pin,0.0,NaN,NaN,NaN,NaN,NaN
Pout,0.0,NaN,NaN,NaN,NaN,NaN


In [11]:
import matplotlib.pyplot as plt


plt.figure(figsize=(15, 15))
Q = [0,
     heaters.loc['PEVD', 'Qg'],
     heaters.loc['PEVD', 'Qg']+heaters.loc['IVD', 'Qg'],
     heaters.loc['PEVD', 'Qg']+heaters.loc['IVD', 'Qg'] +
     heaters.loc['EVD', 'Qg'],
     heaters.loc['PEVD', 'Qg']+heaters.loc['IVD', 'Qg'] +
     heaters.loc['EVD', 'Qg']+heaters.loc['PPND', 'Qg'],
     heaters.loc['PEVD', 'Qg']+heaters.loc['IVD', 'Qg']+heaters.loc['EVD',
                                                                    'Qg']+heaters.loc['PPND', 'Qg']+heaters.loc['IND', 'Qg'],
     heaters.loc['PEVD', 'Qg']+heaters.loc['IVD', 'Qg']+heaters.loc['EVD', 'Qg'] +
     heaters.loc['PPND', 'Qg']+heaters.loc['IND', 'Qg'] +
     heaters.loc['GPK', 'Qg'],
     ]
Q1 = [0,
      heaters.loc['PEVD', 'Qg'],
      heaters.loc['PEVD', 'Qg']+heaters.loc['IVD', 'Qg'],
      heaters.loc['PEVD', 'Qg']+heaters.loc['IVD', 'Qg'] +
      heaters.loc['EVD', 'Qg'],
      heaters.loc['PEVD', 'Qg']+heaters.loc['IVD', 'Qg'] +
      heaters.loc['EVD', 'Qg']+heaters.loc['PPND', 'Qg'],
      heaters.loc['PEVD', 'Qg']+heaters.loc['IVD', 'Qg']+heaters.loc['EVD',
                                                                     'Qg']+heaters.loc['PPND', 'Qg']+heaters.loc['IND', 'Qg'],
      heaters.loc['PEVD', 'Qg']+heaters.loc['IVD', 'Qg']+heaters.loc['EVD', 'Qg'] +
      heaters.loc['PPND', 'Qg']+heaters.loc['IND', 'Qg'] +
      heaters.loc['GPK', 'Qg'],
      ]
Tg = gas_streams.loc['GTU-PEVD':'GPK-out', 'T']
Tw = water_streams.loc['PEVD-DROSVD':'GPK-out', 'REC-GPK']
plt.plot(Q1, Tw, Q, Tg)
plt.xlabel('Q')
plt.ylabel('T')
plt.show()

KeyError: 'REC-GPK'

<Figure size 1080x1080 with 0 Axes>